# 1 Introduction

In linear regression, a **continuous output $y$ (or multiple outputs) is estimated based on a set of input variables $x_1, x_2, ..., x_n$ **. 

The output is written as a function of the input features. Then, the parameters of this function are fitted to the dataset.

One way in which the output can be written as a function of the input, is as follows. The features are $x_1, x_2, ..., x_n$.

$y = w_0 + w_1x_1 + w_2x_2 + ... + w_nx_n$

The parameters $w_0, ... w_n$ can than be fitted to the dataset $y, w_0..w_n$. They will be chosen so that their predictions perform well on the training data. But this function assumes that the relationship between input and output is linear. The output is always a linear combination of the inputs. The learned relationship will be the best possible **linear** relationship.

In the real world, a lot of things are non-linear. If we want our model to be able to learn **non-linear relationships**, we must do **feature engineering**. 

For example, if we suspect that our output might actually depends quadratically on the second input feature $x_2$, we might rewrite our equation as below. Now, the features are $x_1, x_2^2, x_3, ... x_n$ an. We will **make a distinction between variables $x_1, x_2, .. x_n$ and the features that are derived from them**. One feature is one of the variables, or the square of a variable, or even the product of two variables, or any other function of one or more of the variables.

$y = w_0 + w_1x_1 + w_2x_2^2 + ... + w_nx_n$

We will use the notation of **'x' for input variables, and 'h' for features**. As such, we can rewrite our equation as follows:

$y = w_0 + w_1h_1 + w_2h_2 + ... + w_nh_n$

In the example above, $h_2 = x_2^2$

From the input variables, the input features are calculated. Then, the model is trained based on the features. The output is still a **linear function of the features**. Therefore, even though the relation of input **variables** to output could be non-linear, we still call this linear regression. 

In that case, the output will depend more strongly on $x_2$. If, on the other hand, we expect that our output $y$ is only very slightly influenced by $x_2$, we might use the square root of $x_2$ instead.

Feature engineering offers a lot of possibilities. We can always add more features. For example, we could use the following set of features: $x_1, x_2,x_2^2,x_3,x_3^2,x_3^4, x_4, \sqrt(x_4),..., x_5, x_6,... x_n$.

When we do feature engineering and add more variations and new features to the model, the model will be able to learn more complex relationships between the input and the output. However, **beware of overfitting**. If the model is to complex, it could fit very well on the training set but perform badly on new data. Especially when less training data is available, choosing a more simple model could improve performance. The trained model is then likely to generalize better to new data. Simplifying the model can also be done by dropping some input variables.

Feature engineering must be done manually by the data scientists **before** creating the model. Experience and a good understanding of the data will certainly help!



# 2 House Sales dataset

This notebook will guide you through a first example of linear regression. The goal is to train a model that is able to predict the selling price of a house, based on some features. 

The dataset is taken from [Kaggle](https://www.kaggle.com/), an online platform and community where data scientists can share approaches, data sets and take part in machine learning competitions. 

Our House Sales dataset consists of 21613 observations of house prices in King County, USA. It includes home sold between May 2014 and May 2015. The input variables include the living area, number of bathrooms, number of bedrooms, wether or not there is a waterfront view, the condition and grade, building year, ...More info can be found [here](https://www.kaggle.com/harlfoxem/housesalesprediction). 

In what follows, we explore the data and we try to extract useful features from the input variables. Then we train a regression model with the third-party Python library **sklearn**. We will discuss important concepts like the **train-test split** and **over- and underfitting** and see these concepts in action. They are very important for the whole field of machine learning, not only for regression. 

# 3 Feature engineering

## 3.1 Reading and exploring the data

In [55]:
import pandas as pd

In [56]:
dataset = pd.read_csv("kc_house_data.csv")

In [57]:
print("columns: ",dataset.columns)
print("")
print("number of columns: ",len(dataset.columns))
print("number of rows: ",len(dataset))

columns:  Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15'],
      dtype='object')

number of columns:  21
number of rows:  21613


Our dataset consists of 19 columns for the input variables, one column with a house ID and one column with the price (i.e. the target output).

In [58]:
dataset.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [59]:
dataset.dtypes

id                 int64
date              object
price            float64
bedrooms           int64
bathrooms        float64
sqft_living        int64
sqft_lot           int64
floors           float64
waterfront         int64
view               int64
condition          int64
grade              int64
sqft_above         int64
sqft_basement      int64
yr_built           int64
yr_renovated       int64
zipcode            int64
lat              float64
long             float64
sqft_living15      int64
sqft_lot15         int64
dtype: object

## 3.2 One-hot encoding categorical features

The example data we printed above, did not include all columns due to space restrictions. Let's zoom in on some of the columns...

In [60]:
dataset[["waterfront","view","condition","grade","zipcode"]].head(10)

,waterfront,view,condition,grade,zipcode
0,0,0,3,7,98178
1,0,0,3,7,98125
2,0,0,3,6,98028
3,0,0,5,7,98136
4,0,0,3,8,98074
5,0,0,3,11,98053
6,0,0,3,7,98003
7,0,0,3,7,98198
8,0,0,3,7,98146
9,0,0,3,7,98038


First, we must check if any of the data is categorical. Most columns seem numerical, but others are categorical. 

For example, the **zip code**, even though it is a number, is a clear example of a categorical value. There is no real-world relationship between the price of a house and the magnitude of the zipcode. Having a zipcode of 99000 does not mean my house is more expensive than a house with zipcode 98000 or vice versa. 

There are 70 different zipcodes...

In [61]:
print("zipcodes: ",dataset["zipcode"].unique())
print("")
print("number of zipcodes: ",len(dataset["zipcode"].unique()))


zipcodes:  [98178 98125 98028 98136 98074 98053 98003 98198 98146 98038 98007 98115
 98107 98126 98019 98103 98002 98133 98040 98092 98030 98119 98112 98052
 98027 98117 98058 98001 98056 98166 98023 98070 98148 98105 98042 98008
 98059 98122 98144 98004 98005 98034 98075 98116 98010 98118 98199 98032
 98045 98102 98077 98108 98168 98177 98065 98029 98006 98109 98022 98033
 98155 98024 98011 98031 98106 98072 98188 98014 98055 98039]

number of zipcodes:  70


Still, we want to incorporate the zipcode into the model. The location of a house could have an influence on its price. We will **one-hot encode** the zipcode column. This means we add a column to the dataset for each of the 70 zipcodes. For each row, we put a 1 in the column that matches the houses zipcode, and a 0 in the others. Now, the zipcode column is split up into 70 numerical "feature columns".

In [62]:
def encode_categorical(dataframe, column):
    
    #return the same dataframe, but with the column one-hot encoded
    dummies = pd.get_dummies(dataframe[column])
    dummies.columns = [column+str(value) for value in dummies.columns]
    dataframe = dataframe.drop(columns = column)
    for new_column in dummies.columns:
        dataframe[new_column] = dummies[new_column]
    return dataframe
    

In [63]:
dataset = encode_categorical(dataset,'zipcode')

In [64]:
dataset.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,zipcode98146,zipcode98148,zipcode98155,zipcode98166,zipcode98168,zipcode98177,zipcode98178,zipcode98188,zipcode98198,zipcode98199
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,0,0,0,0,0,0,0,0,0,0


Let's zoom into 4 additional variables: waterfront, view, condition, grade. 

For each of these variables, print all the unique values and the number of unique values.

In [65]:
print("waterfront:")
print(dataset["waterfront"].unique())
print(len(dataset["waterfront"].unique()))

print("view:")
print(dataset["view"].unique())
print(len(dataset["view"].unique()))

print("condition:")
print(dataset["condition"].unique())
print(len(dataset["condition"].unique()))

print("grade:")
print(dataset["grade"].unique())
print(len(dataset["grade"].unique()))

waterfront:
[0 1]
2
view:
[0 3 4 2 1]
5
condition:
[3 5 4 1 2]
5
grade:
[ 7  6  8 11  9  5 10 12  4  3 13  1]
12


"waterfront" is a categorical feature, but since it is binary it is already "one-hot-encoded". The "view" property tells us how many times the house has been viewed by potential buyers. A high "view" value - a house that has been viewed a lot - might become more expensive, so this is indeed a numerical value. The "grade" and "condition" values give a score about the condition and grade (according to a King Country grading system) of the house. We will keep these values as numerical, assuming that a high score for either of these values corresponds to a relatively more expense house.

These are the feature columns we have up to now...

In [66]:
dataset.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'lat',
       'long', 'sqft_living15', 'sqft_lot15', 'zipcode98001', 'zipcode98002',
       'zipcode98003', 'zipcode98004', 'zipcode98005', 'zipcode98006',
       'zipcode98007', 'zipcode98008', 'zipcode98010', 'zipcode98011',
       'zipcode98014', 'zipcode98019', 'zipcode98022', 'zipcode98023',
       'zipcode98024', 'zipcode98027', 'zipcode98028', 'zipcode98029',
       'zipcode98030', 'zipcode98031', 'zipcode98032', 'zipcode98033',
       'zipcode98034', 'zipcode98038', 'zipcode98039', 'zipcode98040',
       'zipcode98042', 'zipcode98045', 'zipcode98052', 'zipcode98053',
       'zipcode98055', 'zipcode98056', 'zipcode98058', 'zipcode98059',
       'zipcode98065', 'zipcode98070', 'zipcode98072', 'zipcode98074',
       'zipcode98075', 'zipcode98077', 'zipcode98092', 'zipcode9810

## 3.3 Building age

The 'yr_built' column tells us the building year of the house. The column 'date' contains the date on which the house was sold. By combining these two columns for each data point, we can determine the age of the house at the time of selling. This could be valuable information: an older house is typically worth less than a recently built house.

In the cells below, add a column 'age' to the dataset.

In [67]:
dataset[["date","yr_built"]].head()

,date,yr_built
0,20141013T000000,1955
1,20141209T000000,1951
2,20150225T000000,1933
3,20141209T000000,1965
4,20150218T000000,1987


In [68]:
#step 1: extract the selling YEAR from the 'date' column and store in a new column 'year'.

dataset["selling_year"] = pd.DatetimeIndex(dataset["date"]).year
dataset["selling_year"].head()

0    2014
1    2014
2    2015
3    2014
4    2015
Name: selling_year, dtype: int64

In [69]:
#step 2: subtract yr_built from selling_yr to obtain the building age at selling time.

dataset["age"] = dataset["selling_year"] - dataset["yr_built"]
dataset["age"].head()

0    59
1    63
2    82
3    49
4    28
Name: age, dtype: int64

We also have the feature 'yr_renovated'. A house that was recently renovated, could be more expensive. Similarly to the building age, we could build a column 'years_since_renovation' that tells us how many years before selling there was a renovation. Let's first check out this column.

In [70]:
dataset["yr_renovated"].head(10)

0       0
1    1991
2       0
3       0
4       0
5       0
6       0
7       0
8       0
9       0
Name: yr_renovated, dtype: int64

If the house was never renovated, the year is set to zero. This is a problem, because this means we cannot simply subtract the selling year with the renovation year. Otherwise, a house that was renovated would have a meaningful value for 'years_since_renovation' and one that was never renovated would be "renovated" 2014 or 2015 years ago. Surely, our model would not improve if we added these non-sensical values to the dataset?

More-over, only a small percentage of our houses was renovated at least once. Just how many houses in our dataset were renovated?

In [71]:
# find the percentage of houses that were renovated

(dataset["yr_renovated"] != 0).sum()/len(dataset)

0.042289362883449776

Therefore, we will drop this column. 

**note: **If we wanted to, one way to include the renovation value was one-hot-encoding. We could add a column "renovated" that told us wether or not the building was renovated at least once. We would disregard the exact renovation year, but the feature could still be useful: 


Say $h_n$ is our onehot-encoded 'renovated' feature, we would expect to learn a positive weight $w_n$. Then, $w_nh_n$ could contribute positively to our output price $y$ if the house was once renovated ($h_n > 0$) and would not contribute at all to the price if it was never renovated ($h_n = 0$)


In [72]:
#drop the yr_renovated column. Also drop yr_built and selling_year, since they are no longer needed.

dataset = dataset.drop(columns="yr_renovated")
dataset = dataset.drop(columns="yr_built")
dataset = dataset.drop(columns="selling_year")

dataset.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,zipcode98155,zipcode98166,zipcode98168,zipcode98177,zipcode98178,zipcode98188,zipcode98198,zipcode98199,selling_year,age
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,0,0,0,0,1,0,0,0,2014,59
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,0,0,0,0,0,0,0,0,2014,63
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,0,0,0,0,0,0,0,0,2015,82
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,0,0,0,0,0,0,0,0,2014,49
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,0,0,0,0,0,0,0,0,2015,28


## 3.4 Dropping the date

One might argue that not only the age of time of selling, but also the exact selling date will influence the price. Market prices fluctuate and could be higher at certain times. We have sales data for the period of May 2014 to May 2015. Different scenarios are possible, due to market fluctuations.

1. Market prices are high in May 2014, and have steadily decreased to a lower point in May 2015.
2. Market prices are low in May 2014, and have steadily increased to a higher point in May 2015.
3. Market prices started rising in May 2014, reached a peak, and decreased back to the same level in May 2015.

Let's revisit our equation:

$y = w_0 + w_1h_1 + w_2h_2 + ... + w_nh_n$

We need to be able to find weights (positive or negative) that represent the impact of each feature $h$ on our output price $y$. If $h_1$ is our living area, for example, we expect our learned weight $w_1$ to be positive, so that a high living area results in a high selling price. On the contrary, the weight for the feature 'age' will be negative: an old building is less valuable.
In any case, **we must create features from our input variables that have a uniformly positive or negative influence on the output**. This is a limitation of linear regression. 



In [17]:
dataset["date"].head()

0    20141013T000000
1    20141209T000000
2    20150225T000000
3    20141209T000000
4    20150218T000000
Name: date, dtype: object

So what about our selling date? The timestamp string could be easily transformed into a numerical value. A higher value would mean a recent date, and a lower value would mean an earlier date. But due to market fluctuations, the selling price will not necessarily rise or decrease uniformly in time. This would be true in scenario 1 and 2 above, but not in scenario 3. Since there is no obvious way to transform the date alone into a useful feature, we decide to drop it.




In [75]:
#drop the date column

dataset = dataset.drop(columns="date")

**Note:** other algorithms, notably neural networks, are better in handling non-linear behaviour - even without the need for manual feature engineering. If market prices are higher in the middle of the observed period as in scenario 3, a neural network would be better at capturing this behaviour.

## 3.5 Dropping more columns

Apply a similar reasoning to the longitude and latitude information, as we did for the date column. Should we keep these columns?  

In [80]:
dataset[["lat","long"]].head()

,lat,long
0,47.5112,-122.257
1,47.7210,-122.319
2,47.7379,-122.233
3,47.5208,-122.393
4,47.6168,-122.045


We will drop these columns for a similar reason as the date. Moreover, the house location is already represented with the one-hot-encoded zipcode.

Also drop the useless 'id' column.

In [82]:
dataset["id"].head()

0    7129300520
1    6414100192
2    5631500400
3    2487200875
4    1954400510
Name: id, dtype: int64

In [83]:
#drop lat, long, id

dataset = dataset.drop(columns="lat")
dataset = dataset.drop(columns="long")
dataset = dataset.drop(columns="id")

## 3.6 Resulting dataset

In [84]:
dataset.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,zipcode98148,zipcode98155,zipcode98166,zipcode98168,zipcode98177,zipcode98178,zipcode98188,zipcode98198,zipcode98199,age
0,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,...,0,0,0,0,0,1,0,0,0,59
1,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,...,0,0,0,0,0,0,0,0,0,63
2,180000.0,2,1.00,770,10000,1.0,0,0,3,6,...,0,0,0,0,0,0,0,0,0,82
3,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,...,0,0,0,0,0,0,0,0,0,49
4,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,...,0,0,0,0,0,0,0,0,0,28


In [85]:
dataset.columns

Index(['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'sqft_living15', 'sqft_lot15', 'zipcode98001',
       'zipcode98002', 'zipcode98003', 'zipcode98004', 'zipcode98005',
       'zipcode98006', 'zipcode98007', 'zipcode98008', 'zipcode98010',
       'zipcode98011', 'zipcode98014', 'zipcode98019', 'zipcode98022',
       'zipcode98023', 'zipcode98024', 'zipcode98027', 'zipcode98028',
       'zipcode98029', 'zipcode98030', 'zipcode98031', 'zipcode98032',
       'zipcode98033', 'zipcode98034', 'zipcode98038', 'zipcode98039',
       'zipcode98040', 'zipcode98042', 'zipcode98045', 'zipcode98052',
       'zipcode98053', 'zipcode98055', 'zipcode98056', 'zipcode98058',
       'zipcode98059', 'zipcode98065', 'zipcode98070', 'zipcode98072',
       'zipcode98074', 'zipcode98075', 'zipcode98077', 'zipcode98092',
       'zipcode98102', 'zipcode98103', 'zipcode98105', 'zipcode98106',
     

# 4 Training the model

Our data is now finally ready. Let's train the model using the third-party library **sklearn**. 

[documentation for the sklearn linear regressor](http://scikit-learn.org/dev/modules/generated/sklearn.linear_model.LinearRegression.html)

In [86]:
! pip install sklearn

In [100]:
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [89]:
#split the dataset in two pandas dataframe: one for all the features, another for just the price (our target)

dataset_target = dataset["price"]
dataset_features = dataset.drop(columns="price")

In [92]:
#The fitting method of our regressor will require a numpy array as input. Convert the data inside the pandas dataframe to a numpy array.

H = dataset_features.values
Y = dataset_target.values

In [93]:
#Check the dimensions of these arrays

print(H.shape)
print(Y.shape)

(21613, 84)
(21613,)


We split the data into a training set and a testing set. First we **train the model on the training data**, then we will **evaluate the performance of the model on the test set**. 

This way, we can determine wether or not the model is able to generalize to new, unseen data. A high performance on the training data does not mean you have a valuable model. Maybe the parameters were optimized to perform very well on the training data during training, but it does not perform well on new data. 

**note: ** We will later see that the train-test split (or even train-test-dev split) is also used for hyperparameter tuning. This is not relevant here since we have no hyperparameters to tune.

In [101]:
#Train-test split

X_train, X_test, Y_train, Y_test = train_test_split(H,Y,test_size = 0.10)

In [102]:
regressionModel = LinearRegression()

In [103]:
regressionModel.fit(X_train,Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [104]:
regressionModel.score(X_test, Y_test)

0.7998547502322202

# 5 Extra feature engineering and training

# 6 Conclusion and take-aways